In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.ensemble import *

In [4]:
deliveries = pd.read_csv("deliveries.csv")
matches = pd.read_csv("matches.csv")

In [5]:
df = deliveries.groupby(by=["match_id","batting_team","bowling_team"])["total_runs"].sum()
df = pd.DataFrame(df).reset_index().set_index("match_id")


scores = pd.DataFrame(columns=["match_id","Teams","Toss Winner","Toss Decision","Team 1","Team 2","Team 1 Score","Team 2 Score","Team 1 Wickets","Team 2 Wickets"])
team_scores = {}
ctr = 0
for i in df.index.unique():
    #print(df.loc[i,"batting_team"])
    try:
        teams = df.loc[i,"batting_team"].values
        toss_winner = matches[matches["id"]==i]["toss_winner"].values[0]
        choice = matches[matches["id"]==i]["toss_decision"].values[0]
        #print("Toss Winning Team : ",toss_winner)
        #print("Choice of",toss_winner,":",choice)

        position_of_toss_winner = np.where(teams == toss_winner)[0][0]
        toss_win = teams[position_of_toss_winner]

        if toss_winner == teams[0]:
            if choice == "bat":
                first_batting_team = teams[0]
            elif choice == "field":
                first_batting_team = teams[1]
        else:
            if choice == "bat":
                first_batting_team = teams[1]
            elif choice == "field":
                first_batting_team = teams[0]
        second_batting_team = teams[1-np.where(teams==first_batting_team)[0][0]]
        p = teams[0] + " vs " + teams[1]
        scores.loc[ctr,"match_id"] = i
        scores.loc[ctr,"Teams"] = p
        scores.loc[ctr,"Team 1"] = first_batting_team
        scores.loc[ctr,"Team 2"] = second_batting_team
        particular_match = df.loc[i,]
        scores.loc[ctr,"Team 1 Score"] = particular_match[particular_match["batting_team"]==first_batting_team]["total_runs"].values[0]
        scores.loc[ctr,"Team 2 Score"] = particular_match[particular_match["batting_team"]==second_batting_team]["total_runs"].values[0]
        scores.loc[ctr,"Toss Winner"] = toss_winner
        scores.loc[ctr,"Toss Decision"] = choice
        #print(p)
        ctr += 1
        #break
    #print()
    except:
        batting_team = df.loc[i,"batting_team"]
        bowling_team = df.loc[i,"bowling_team"]

        toss_winner = matches[matches["id"]==i]["toss_winner"].values[0]
        choice = matches[matches["id"]==i]["toss_decision"].values[0]
        #print("Toss Winning Team : ",toss_winner)
        #print("Choice of",toss_winner,":",choice)

        #position_of_toss_winner = np.where(teams == toss_winner)[0][0]
        #toss_win = teams[position_of_toss_winner]
        
        scores.loc[ctr,"match_id"] = i
        scores.loc[ctr,"Teams"] = batting_team + " vs " + bowling_team
        scores.loc[ctr,"Team 1"] = batting_team
        scores.loc[ctr,"Team 2"] = bowling_team
        particular_match = df.loc[i,]
        scores.loc[ctr,"Team 1 Score"] = particular_match["total_runs"]
        scores.loc[ctr,"Team 2 Score"] = np.nan
        scores.loc[ctr,"Toss Winner"] = toss_winner
        scores.loc[ctr,"Toss Decision"] = choice
        ctr += 1

In [6]:
wickets = pd.DataFrame(columns=["Team 1 Wickets","Team 2 Wickets"])
ctr = 0
for i in deliveries["match_id"].unique():
    wickets_first_team = deliveries[deliveries["match_id"]==i][deliveries["inning"]==1]["player_dismissed"].count()
    wickets.loc[ctr,"Team 1 Wickets"] = wickets_first_team
    try:
        wickets_second_team = deliveries[deliveries["match_id"]==i][deliveries["inning"]==2]["player_dismissed"].count()
        wickets.loc[ctr,"Team 2 Wickets"] = wickets_second_team
    except:
        print()
    ctr += 1

scores["Team 1 Wickets"] = wickets["Team 1 Wickets"].copy()
scores["Team 2 Wickets"] = wickets["Team 2 Wickets"].copy()
scores["city"] = matches["city"].copy()


In [7]:
deli = deliveries[["match_id","inning","over","ball"]].drop_duplicates()
deli = deli[deli["ball"]<=6].groupby(by=["match_id","inning"])["ball"].count()
deli = deli.reset_index()
dgh = pd.DataFrame(columns=["match_id","Team 1 Overs","Team 1 Balls","Team 2 Overs","Team 2 Balls"])
ctr = 0
for i in deli["match_id"].unique():
    dgh.loc[ctr,"match_id"] = i

    total_balls_first_team = deli[deli["match_id"]==i]["ball"].values[0]

    overs_first_team = int(np.floor(total_balls_first_team / 6))
    balls_first_team = total_balls_first_team - overs_first_team * 6

    dgh.loc[ctr,"Team 1 Overs"] = overs_first_team
    dgh.loc[ctr,"Team 1 Balls"] = balls_first_team
    

    try:
        total_balls_second_team = deli[deli["match_id"]==i]["ball"].values[1]

        overs_second_team = int(np.floor(total_balls_second_team / 6))
        balls_second_team = total_balls_second_team - overs_second_team * 6

        dgh.loc[ctr,"Team 2 Overs"] = overs_second_team
        dgh.loc[ctr,"Team 2 Balls"] = balls_second_team
    except:
        print()
    ctr += 1

dgh.head()

scores[["Team 1 Overs","Team 1 Balls","Team 2 Overs","Team 2 Balls"]] = dgh[["Team 1 Overs","Team 1 Balls","Team 2 Overs","Team 2 Balls"]].copy()

scores["Target"] = scores["Team 1 Score"] + 1
scores.head()

,match_id,Teams,Toss Winner,Toss Decision,Team 1,Team 2,Team 1 Score,Team 2 Score,Team 1 Wickets,Team 2 Wickets,city,Team 1 Overs,Team 1 Balls,Team 2 Overs,Team 2 Balls,Target
0,1,Royal Challengers Bangalore vs Sunrisers Hyder...,Royal Challengers Bangalore,field,Sunrisers Hyderabad,Royal Challengers Bangalore,207,172,4,10,Hyderabad,20,0,19,4,208
1,2,Mumbai Indians vs Rising Pune Supergiant,Rising Pune Supergiant,field,Mumbai Indians,Rising Pune Supergiant,184,187,8,3,Pune,20,0,19,5,185
2,3,Gujarat Lions vs Kolkata Knight Riders,Kolkata Knight Riders,field,Gujarat Lions,Kolkata Knight Riders,183,184,4,0,Rajkot,20,0,15,0,184
3,4,Kings XI Punjab vs Rising Pune Supergiant,Kings XI Punjab,field,Rising Pune Supergiant,Kings XI Punjab,163,164,6,4,Indore,20,0,19,0,164
4,5,Delhi Daredevils vs Royal Challengers Bangalore,Royal Challengers Bangalore,bat,Royal Challengers Bangalore,Delhi Daredevils,157,142,8,9,Bangalore,20,0,20,0,158


In [15]:
df_1.head()

,Team 1,Team 2,city,Target,Team 2 Score,Team 2 Overs,Team 2 Balls,Team 2 Wickets
0,Sunrisers Hyderabad,Royal Challengers Bangalore,Hyderabad,208,172,19,4,10
1,Mumbai Indians,Rising Pune Supergiant,Pune,185,187,19,5,3
2,Gujarat Lions,Kolkata Knight Riders,Rajkot,184,184,15,0,0
3,Rising Pune Supergiant,Kings XI Punjab,Indore,164,164,19,0,4
4,Royal Challengers Bangalore,Delhi Daredevils,Bangalore,158,142,20,0,9


In [46]:
df_1 = scores[["Team 1", "Team 2", "city", "Target", "Team 2 Score", "Team 2 Overs", "Team 2 Balls", "Team 2 Wickets"]]



int_columns = ["Target", "Team 2 Score", "Team 2 Overs", "Team 2 Balls", "Team 2 Wickets"]

for i in int_columns:
    df_1[i] = df_1[i].fillna(0)
    df_1[i] = df_1[i].astype("int64")

df_1["city"] = df_1["city"].fillna("Not specified")

In [49]:
ohe = OneHotEncoder(drop="first")
#print(df_1.dtypes)

cols = df_1.dtypes[df_1.dtypes=="object"].index.values.tolist()

df_2 = df_1.copy()


one_hot_encoded = ohe.fit_transform(df_2[cols])

df_3 = pd.DataFrame(ohe.fit_transform(df_2[cols]).toarray(),columns=ohe.get_feature_names_out(cols))

df_3[int_columns] = df_2[int_columns]

df_3.head()

,Team 1_Deccan Chargers,Team 1_Delhi Capitals,Team 1_Delhi Daredevils,Team 1_Gujarat Lions,Team 1_Kings XI Punjab,Team 1_Kochi Tuskers Kerala,Team 1_Kolkata Knight Riders,Team 1_Mumbai Indians,Team 1_Pune Warriors,Team 1_Rajasthan Royals,...,city_Raipur,city_Rajkot,city_Ranchi,city_Sharjah,city_Visakhapatnam,Target,Team 2 Score,Team 2 Overs,Team 2 Balls,Team 2 Wickets
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,208,172,19,4,10
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,185,187,19,5,3
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,184,184,15,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,164,164,19,0,4
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,158,142,20,0,9


In [54]:
X = df_3.drop("Target",axis=1)
y = df_3["Target"]

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=np.random.randint(0,1000),test_size=0.2)

